# Convert DEIMOS spectra to PFS datamodel FITS files

In [0]:
PROJECT_PATH = [
    '/home/dobos/project/Subaru-PFS/datamodel/python',
    '/home/dobos/project/Subaru-PFS/psf_utils/python'
]

DATA_PATH = '/datascope/subaru/data/catalogs/deimos/deimos_gc_spectra_mags.h5'
OUTPUT_PATH = '/datascope/subaru/user/dobos/data/pfsspec/mock/deimos'

ARMS = [ 'r' ]

In [0]:
import os
from glob import glob
import sys
import re
import logging
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
from scipy.interpolate import interp1d
import h5py as h5
from tqdm.notebook import trange, tqdm
from collections.abc import Iterable
import pickle
import debugpy
import cProfile, pstats

In [0]:
plt.rc('font', size=7)

In [0]:
# Allow load project as module
for p in reversed(PROJECT_PATH):
    sys.path.insert(0, p)

In [0]:
%load_ext autoreload

In [0]:
%autoreload 2

In [0]:
if 'debug' not in globals():
    import debugpy
    debugpy.listen(("localhost", 5683))
    debug = True

## Load Deimos spectra from HDF5

In [0]:
deimos = h5.File(DATA_PATH, 'r')

In [0]:
def print_hdf5_structure(g, prefix=''):
    for k in g:
        if isinstance(g[k], h5.Group):
            print(prefix + k)
            print_hdf5_structure(g[k], prefix + '  ')
        elif isinstance(g[k], h5.Dataset):
            print(prefix + k, g[k].shape, g[k].dtype)

print_hdf5_structure(deimos)

In [0]:
from pfs.ga.pfsspec.core import Spectrum
from pfs.ga.pfsspec.core import Physics

In [0]:
def get_spectrum(i):
    # Read spectrum into pfsspec object

    spec = Spectrum()
    spec.id = i
    spec.name = str(deimos['objname'][i])
    spec.mjd = deimos['mjd'][i]

    spec.wave = deimos['spectra']['rest_wvl'][i, :]
    # spec.flux = deimos['spectra']['flux'][i, :]
    # spec.flux_err = deimos['spectra']['ivar'][i, :] ** (-1 / 2)
    spec.mask = deimos['spectra']['mask'][i, :].astype(int)
    spec.flux = deimos['spectra']['telldiv'][i, :]
    spec.flux_err = deimos['spectra']['telldivivar'][i, :] ** (-1 / 2)

    spec.Fe_H = spec.mjd = deimos['measurements']['feh'][i]
    spec.Fe_H_err = spec.mjd = deimos['measurements']['feherr'][i]
    spec.log_g = spec.mjd = deimos['measurements']['logg'][i]
    spec.log_g_err = spec.mjd = deimos['measurements']['loggerr'][i]
    spec.T_eff = spec.mjd = deimos['measurements']['teff'][i]
    spec.T_eff_err = spec.mjd = deimos['measurements']['tefferr'][i]
    spec.a_Fe = spec.mjd = deimos['measurements']['alphafe'][i]
    spec.a_Fe_err = spec.mjd = deimos['measurements']['alphafeerr'][i]
    spec.Ca_Fe = spec.mjd = deimos['measurements']['cafe'][i]
    spec.Ca_Fe_err = spec.mjd = deimos['measurements']['cafeerr'][i]
    spec.Mg_Fe = spec.mjd = deimos['measurements']['mgfe'][i]
    spec.Mg_Fe_err = spec.mjd = deimos['measurements']['mgfeerr'][i]
    spec.Ni_Fe = spec.mjd = deimos['measurements']['nife'][i]
    spec.Ni_Fe_err = spec.mjd = deimos['measurements']['nifeerr'][i]
    spec.Si_Fe = spec.mjd = deimos['measurements']['sife'][i]
    spec.Si_Fe_err = spec.mjd = deimos['measurements']['sifeerr'][i]
    spec.Ti_Fe = spec.mjd = deimos['measurements']['tife'][i]
    spec.Ti_Fe_err = spec.mjd = deimos['measurements']['tifeerr'][i]
    spec.rv = spec.mjd = deimos['measurements']['vhelio'][i]
    spec.rv_err = spec.mjd = deimos['measurements']['vhelioerr'][i]

    return spec


In [0]:
s = get_spectrum(10)

plt.plot(s.wave, s.flux, lw=0.3)
plt.plot(s.wave, s.flux_err, lw=0.3)

plt.title(s.name)

In [0]:
s.mask

## Generate pfsSingle files

In [0]:
from pfs.ga.pfsspec.core import Physics

In [0]:
from pfs.datamodel import Target, Identity, Observations, MaskHelper
from pfs.datamodel import PfsArm, PfsObject, PfsConfig, TargetType, FiberStatus, GuideStars
from pfs.datamodel import PfsSingle, PfsMerged
from pfs.datamodel import FluxTable
from pfs.datamodel.utils import calculatePfsVisitHash

In [0]:
designid = 7884270544754596914
designname = 'deimos'
arms = ''.join([arm[0] for arm in ARMS])

In [0]:
N = 10

In [0]:
visit = 98765
date = '2024-01-03'

raboresight = 0.0
decboresight = 0.0
posang = 0.0

fiberid = np.arange(N)          # Just incremental numbers to fake fiber ids
tract = np.full((N,), 1)
patch = np.full((N,), '1,1')

In [0]:
catid = []
tract = []
patch = []
objid = []
ra = []
dec = []
targettype = np.full((N,), TargetType.SCIENCE)
fiberStatus = np.full((N,), FiberStatus.GOOD)
epoch = np.full((N,), 'J2000.0')
pmra = np.full((N,), 0.0)
pmdec = np.full((N,), 0.0)
parallax = np.full((N,), 1e-8)
fiberflux = []

proposalid = []
obcode = []

spectrograph = []
fiberid = []
pfinominal = []
pficenter = []

In [0]:
for i in range(N):
    spec = { 'r': get_spectrum(i) }
    
    catid.append(0)                           # Simulated catalog
    tract.append(0)
    patch.append('1,1')
    objid.append(i)
    ra.append(0.0)                            # spec[ARMS[0]].ra
    dec.append(0.0)                           # spec[ARMS[0]].dec
    
    fiberflux_dict = {}
    for m in ['B', 'H', 'I', 'J', 'K', 'R', 'V']:
        fiberflux_dict[m] = Physics.abmag_to_jy(deimos['photometry'][m][i])  # assumes 100% flux in the fiber
    fiberflux.append(fiberflux_dict)

    proposalid.append('N/A')
    obcode.append('N/A')

    target = Target(catid[i], tract[i], patch[i], objid[i], ra[i], dec[i], targettype[i], fiberflux[i])
    target.identity["visit"] = visit        # For PfsSingle only
    
    visit_array = np.array([visit])           # Single exposure
    spectrograph.append(np.array([0]))
    fiberid.append(np.array([i % 2400]))
    pfinominal.append(np.array([[0.0, 0.0]]))
    pficenter.append(np.array([[0.0, 0.0]]))

    observations = Observations(visit_array, np.array([arms]), spectrograph[i], np.array([designid]), fiberid[i], pfinominal[i], pficenter[i])

    # Combine arrays
    # TODO: don't just concatenate but resample similar to real pfsSingle files

    # A to nm
    wave = np.concatenate([ 1e-1 * spec[arm].wave for arm in ARMS ])

    # Flux and error (sigma)
    flux = np.concatenate([ spec[arm].flux for arm in ARMS])
    error = np.concatenate([ spec[arm].flux_err for arm in ARMS])
    
    # Add noise
    flux = flux + error * np.random.normal(size=flux.shape)
    
    # erg/sec/cm2/A to nJy
    flux = 10e32 * Physics.flam_to_fnu(wave * 10, flux)
    error = 10e32 * Physics.flam_to_fnu(wave * 10, error)

    # TODO: save sky model into dataset
    sky = np.zeros_like(flux)

    # We assume diagonal here but it has to be tridiagonal so just pad the array
    # Sigma of simulated spectra is given in same units as flux
    covar = np.pad(error[None, :] ** 2, ((1, 1), (0, 0)))
    covar2 = np.array([[0.0]])

    # TODO: use PFS mask bits
    mask = np.concatenate([ spec[arm].mask_as_int() for arm in ARMS])

    # PfsSpec flags -- these are 64 bit
    # flags = MaskHelper(
    #     **{ k[4:]: int(v) for k, v in Spectrum.__dict__.items() if k.startswith('MASK_')}    
    # )

    # PFS datamodel flags -- these are 32 bit
    flags = MaskHelper(
        **{'BAD': 0,
           'BAD_FIBERTRACE': 11,
           'BAD_FLAT': 9,
           'BAD_FLUXCAL': 13,
           'BAD_SKY': 12,
           'CR': 3,
           'DETECTED': 5,
           'DETECTED_NEGATIVE': 6,
           'EDGE': 4,
           'FIBERTRACE': 10,
           'INTRP': 2,
           'IPC': 14,
           'NO_DATA': 8,
           'REFLINE': 15,
           'SAT': 1,
           'SUSPECT': 7,
           'UNMASKEDNAN': 16}    
    )

    # These are always the non-resampled values
    fluxtable = FluxTable(wave, flux, error, mask, flags=flags)
    
    single = PfsSingle(target, observations, wave, flux, mask, sky,
                          covar, covar2,
                          flags=flags,
                          metadata=None, fluxTable=fluxtable, notes=None)

    # print(pfssingle.observervations)

    # Filename is composed by the library but not the path
    dir = os.path.join(OUTPUT_PATH, f'pfsSingle/{catid[i]:05d}/{tract[i]:05d}/{patch[i]}')
    
    if not os.path.isdir(dir):
        os.makedirs(dir, exist_ok=True)
    
    print(dir)
    print(single.filenameFormat % single.getIdentity())
    
    single.write(dirName=dir)


In [0]:
guidestars = GuideStars(objId=np.array([0]),
                        epoch=np.array(['J2016.0']),
                        ra=np.array([0.0]),
                        dec=np.array([0.0]),
                        pmRa=np.array([0.0]),
                        pmDec=np.array([0.0]),
                        parallax=np.array([0.0]),
                        magnitude=np.array([0.0]),
                        passband=np.array(['g_gaia']),
                        color=np.array([0.0]),
                        agId=np.array([0]),
                        agX=np.array([0.0]),
                        agY=np.array([0.0]),
                        telElev=0.0,
                        guideStarCatId=0)

In [0]:
fiberflux

In [0]:
# TODO: Do not assume all flux is in the fiber

config = PfsConfig(designid, visit, raboresight, decboresight, posang, arms=arms,
                   fiberId=np.array(fiberid), tract=np.array(tract), patch=np.array(patch),
                   ra=np.array(ra), dec=np.array(dec),
                   catId=np.array(catid), objId=np.array(objid), targetType=np.array(targettype),
                   fiberStatus=np.array(fiberStatus),
                   epoch=np.array(epoch), pmRa=pmra, pmDec=pmdec, parallax=parallax,
                   proposalId=np.array(proposalid),
                   obCode=np.array(obcode),
                   fiberFlux=[ [ f for _, f in ff.items() ] for ff in fiberflux ],
                   psfFlux=[ [ f for _, f in ff.items() ] for ff in fiberflux ],
                   totalFlux=[ [ f for _, f in ff.items() ] for ff in fiberflux ],
                   fiberFluxErr=[ [ 0.0 for _, f in ff.items() ] for ff in fiberflux ],
                   psfFluxErr=[ [ 0.0 for _, f in ff.items() ] for ff in fiberflux ],
                   totalFluxErr=[ [ 0.0 for _, f in ff.items() ] for ff in fiberflux ],
                   filterNames=[ [ n for n, _ in ff.items() ] for ff in fiberflux ],
                   pfiCenter=np.stack(pficenter).squeeze(),
                   pfiNominal=np.stack(pfinominal).squeeze(),
                   guideStars=guidestars,
                   designName=designname
                   )

# Filename is composed by the library but not the path
dir = os.path.join(OUTPUT_PATH, f'pfsConfig/{date}')

if not os.path.isdir(dir):
    os.makedirs(dir, exist_ok=True)

print(dir)
print(config.filename)

config.write(dirName=dir)

In [0]:
spec['mr'].__dict__